# network 2

In [3]:
import numpy as np
import scipy.io
from sklearn.model_selection import train_test_split

In [4]:
class FCLayer:
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

In [5]:
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [6]:
def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

In [7]:
# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

In [8]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [102]:

"""solve xor"""

# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 8))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(8, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=10, learning_rate=0.1)

# test
out = net.predict(x_train)
# print(out)

epoch 1/10   error=0.565484
epoch 2/10   error=0.302764
epoch 3/10   error=0.295053
epoch 4/10   error=0.292119
epoch 5/10   error=0.289768
epoch 6/10   error=0.287575
epoch 7/10   error=0.285454
epoch 8/10   error=0.283377
epoch 9/10   error=0.281329
epoch 10/10   error=0.279301


In [138]:
path = "D:\\work\\university\\Huni_projects\\computional_intelligence\\tasks\\MLP_project\\Data.mat"
mat = scipy.io.loadmat(path)
input_, target_ = mat["Input"], mat["Target"]

# fix data shape
fixed_shape_input = [[input_[0][i], input_[1][i]] for i in range(input_.shape[1])]
X = np.array(fixed_shape_input)
y = np.array(target_[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4321)

# X_train = np.array([[[-2.08796522, -4.4014648 ]],
#                     [[-0.63371396,  2.09373378]],
#                     [[ 2.69171633, -3.35786877]],
#                     [[-3.19845804,  4.52217065]],])

# y_train =np.array([ [[1]], [[-1]], [[-1]], [[-1]] ])

n = y_train.size
y_ = []
X_ = []
for i in range(n):
    y_tmp = np.array([[y_train[i]]])
    x_tmp = np.array([X_train[i]])
    y_.append(y_tmp)
    X_.append(x_tmp)
    
X_train = X_
y_train = y_
    


# network
net = Network()
net.add(FCLayer(2, 2))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(2, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(X_train, y_train, epochs=20, learning_rate=0.01)

# test
# out = net.predict(X_train)
# print(out)


epoch 1/20   error=0.500835
epoch 2/20   error=0.161584
epoch 3/20   error=0.156993
epoch 4/20   error=0.156503
epoch 5/20   error=0.154383
epoch 6/20   error=0.152788
epoch 7/20   error=0.154088
epoch 8/20   error=0.151892
epoch 9/20   error=0.148792
epoch 10/20   error=0.147703
epoch 11/20   error=0.145996
epoch 12/20   error=0.144087
epoch 13/20   error=0.143017
epoch 14/20   error=0.143085
epoch 15/20   error=0.143351
epoch 16/20   error=0.143398
epoch 17/20   error=0.143415
epoch 18/20   error=0.143574
epoch 19/20   error=0.143817
epoch 20/20   error=0.143788
